# Cross-Country Solar Potential Comparison
Analyze solar potential (GHI, DNI, DHI) across Benin, Sierra Leone, and Togo using cleaned datasets.

In [4]:
# Import libraries for data analysis and visualization
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kruskal
import os

# Define project paths
PROJECT_ROOT = r'C:\Users\tsion\OneDrive\Desktop\solar-challenge-week1'
OUTPUT_DIR = 'notbook/notbook_compare_countries_output'

# Create output directory and set working directory
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.chdir(PROJECT_ROOT)
print(f'Working directory: {os.getcwd()}')

# load the dataset
def load_cleaned_data():
    """Load and combine cleaned CSV files with a Country column."""
    # Load each country's data
    benin = pd.read_csv('data/benin_clean.csv')
    togo = pd.read_csv('data/togo_clean.csv')
    sierra = pd.read_csv('data/sierraleone_clean.csv')

    # Add Country identifier
    benin['Country'] = 'Benin'
    togo['Country'] = 'Togo'
    sierra['Country'] = 'Sierra Leone'

    # Combine datasets
    combined = pd.concat([benin, togo, sierra], ignore_index=True)
    print(f'Dataset size: {combined.shape} rows, {combined.columns.size} columns')
    return combined

# Load data
solar_data = load_cleaned_data()

Working directory: C:\Users\tsion\OneDrive\Desktop\solar-challenge-week1
Dataset size: (1576800, 20) rows, 20 columns


## Create Boxplots

Generate boxplots to compare GHI, DNI, and DHI distributions across countries.

In [5]:
def create_boxplots(data, metrics, output_dir):
    """Create boxplots for each metric, colored by country."""
    for metric in metrics:
        plt.figure(figsize=(8, 6))
        sns.boxplot(x='Country', y=metric, data=data, palette='Set2')
        plt.title(f'{metric} Distribution by Country', fontsize=12)
        plt.xlabel('Country', fontsize=10)
        plt.ylabel(f'{metric} (W/m²)', fontsize=10)
        plt.tight_layout()
        plt.savefig(f'{output_dir}/{metric}_boxplot.png', dpi=300)
        plt.close()
        print(f'Generated: {output_dir}/{metric}_boxplot.png')

# Plot boxplots
metrics = ['GHI', 'DNI', 'DHI']
create_boxplots(solar_data, metrics, OUTPUT_DIR)

C:\Users\tsion\AppData\Local\Temp\ipykernel_26284\3408937755.py:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='Country', y=metric, data=data, palette='Set2')


Generated: notbook/notbook_compare_countries_output/GHI_boxplot.png


C:\Users\tsion\AppData\Local\Temp\ipykernel_26284\3408937755.py:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='Country', y=metric, data=data, palette='Set2')


Generated: notbook/notbook_compare_countries_output/DNI_boxplot.png


C:\Users\tsion\AppData\Local\Temp\ipykernel_26284\3408937755.py:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.boxplot(x='Country', y=metric, data=data, palette='Set2')


Generated: notbook/notbook_compare_countries_output/DHI_boxplot.png


## Summary Table

Calculate mean, median, and standard deviation for GHI, DNI, and DHI by country

In [6]:
def generate_summary_table(data, metrics, output_dir):
    """Compute summary statistics and save as CSV."""
    summary = data.groupby('Country')[metrics].agg(['mean', 'median', 'std']).round(2)
    summary.columns = [f'{metric}_{stat}' for metric, stat in summary.columns]
    summary.to_csv(f'{output_dir}/summary_table.csv')
    print('Summary Table:')
    print(summary)
    return summary

# Create summary table
summary_stats = generate_summary_table(solar_data, metrics, OUTPUT_DIR)

Summary Table:
              GHI_mean  GHI_median  GHI_std  DNI_mean  DNI_median  DNI_std  \
Country                                                                      
Benin           239.98         1.8   329.68    166.93        -0.1   261.02   
Sierra Leone    203.55         0.3   294.18    116.10         0.0   217.20   
Togo            231.01         2.1   319.84    150.90         0.0   249.93   

              DHI_mean  DHI_median  DHI_std  
Country                                      
Benin           114.96         1.6   157.43  
Sierra Leone    115.76         0.0   155.42  
Togo            116.01         2.5   155.17  


## Statistical Test

Use the Kruskal-Wallis test to check if GHI differences between countries are significant.

In [7]:
def run_kruskal_wallis(data, metric='GHI'):
    """Perform Kruskal-Wallis test on a metric."""
    groups = [
        data[data['Country'] == country][metric].dropna()
        for country in ['Benin', 'Togo', 'Sierra Leone']
    ]
    stat, p_value = kruskal(*groups)
    print(f'Kruskal-Wallis Test for {metric}:')
    print(f'  Statistic: {stat:.2f}')
    print(f'  P-value: {p_value:.4f}')
    print('  Interpretation:', 'Significant differences (p < 0.05)' if p_value < 0.05 else 'No significant differences (p >= 0.05)')

# Test GHI
run_kruskal_wallis(solar_data, 'GHI')

Kruskal-Wallis Test for GHI:
  Statistic: 53816.36
  P-value: 0.0000
  Interpretation: Significant differences (p < 0.05)


## Rank Countries by GHI

Create a bar chart ranking countries by their average GHI.

In [8]:
def plot_ghi_ranking(data, output_dir):
    """Plot a bar chart of average GHI by country."""
    avg_ghi = data.groupby('Country')['GHI'].mean().sort_values(ascending=False)
    plt.figure(figsize=(8, 6))
    sns.barplot(x=avg_ghi.values, y=avg_ghi.index, palette='Set2')
    plt.title('Average GHI Ranking by Country', fontsize=12)
    plt.xlabel('Mean GHI (W/m²)', fontsize=10)
    plt.ylabel('Country', fontsize=10)
    plt.tight_layout()
    plt.savefig(f'{output_dir}/ghi_ranking.png', dpi=300)
    plt.close()
    print(f'Generated: {output_dir}/ghi_ranking.png')

# Plot ranking
plot_ghi_ranking(solar_data, OUTPUT_DIR)

C:\Users\tsion\AppData\Local\Temp\ipykernel_26284\3560873085.py:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(x=avg_ghi.values, y=avg_ghi.index, palette='Set2')


Generated: notbook/notbook_compare_countries_output/ghi_ranking.png


## Observations

- **Benin exhibits the highest solar potential**, with a mean GHI of 239.98 W/m² and DNI of 166.93 W/m², making it the best candidate for photovoltaic (PV) and concentrated solar power (CSP) installations. The boxplot shows a higher median GHI (around 400-500 W/m²) and significant variability (GHI std = 329.68), suggesting the need for energy storage to handle fluctuations, especially with outliers reaching 1000 W/m².
- **Sierra Leone shows the lowest solar irradiance**, with a mean GHI of 203.55 W/m² and DNI of 116.1 W/m², indicating less suitability for large-scale solar projects. Its lower GHI variability (std = 294.18) and boxplot median (around 300-400 W/m²) suggest more stable but reduced solar conditions, potentially due to cloud cover or humidity.
- **Togo offers a competitive middle ground**, with a mean GHI of 231.01 W/m² and the highest DHI (116.01 W/m²), suitable for diffuse-light-dependent systems like thin-film PV. The Kruskal-Wallis test (p-value = 0.0000) confirms significant GHI differences across countries, and Togo’s boxplot shows a robust median (around 400-500 W/m²) with moderate variability (std = 319.84), indicating consistent solar potential.